# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
import re
import numpy as np
import pandas as pd
from time import time
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix, precision_score,recall_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import sqlite3
from sqlalchemy import create_engine
from pathlib import Path
import pickle

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifierCV
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
import multiprocessing
print(multiprocessing.cpu_count())

1


In [28]:
# load data from database
conn = sqlite3.connect('ETLPipelineDatabase.db')
engine = create_engine('sqlite:///ETLPipelineDatabase.db')
df = pd.read_sql('select * from ETLPipeline_Table',con=conn)
df.drop(['level_0','index'],axis=1,inplace=True)
X = df['message'].values # text array
Y = df.iloc[:,4:].values # text array

In [29]:
print(X[0],X.shape)
print (Y[0],Y.shape)

Weather update - a cold front from Cuba that could pass over Haiti (26042,)
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] (26042, 36)


In [30]:
df['genre'].value_counts()#Categorial values

news      12901
direct    10747
social     2394
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [31]:
##Trial code
# print(X[100])
# text = re.sub(r"[^a-zA-Z0-9]", " ", X[100].lower())
# print(text)
# #text = text.split()
# words = word_tokenize(text)
# print(words)
# # Remove stop words
# words = [w for w in words if w not in stopwords.words("english")]
# print(words)
# # Reduce words to their root form
# lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
# print(lemmed)

    Function tokenize that takes in a string of text and applies the following:

    case normalization (convert to all lowercase)
    punctuation removal
    tokenization, lemmatization, and stop word removal using nltk

In [32]:
def tokenize(text):
    # Remove punctuation characters
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('MOC', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
from sklearn import cross_validation

In [35]:
def display_results(y_test, y_pred):
    """
    Function to display single output single target output.
    """
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("Precision score:", precision_score(y_test, y_pred))
    print("Recall score:", recall_score(y_test, y_pred))

In [36]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)

##train classifier

start = time()

print('Training..........')
pipeline.fit(X_train,y_train)
print('Named_steps: {}'.format(pipeline.named_steps),
      'Score: {}'.format (pipeline.score),
      'Get Params: {}'.format (pipeline.get_params),sep='\n')
#pipeline.steps


print('Time taken: {}'.format(time()-start))
print('....................finished training')

Training..........
Named_steps: {'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f63e1aee158>, vocabulary=None), 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), 'MOC': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),


In [37]:
# predict on test data
print('Predicting..........')
y_pred = pipeline.predict(X_test)
print('..................................finished predicting')

Predicting..........
..................................finished predicting


In [38]:
print('Evaluating ...........')
display_results(y_test[:,1], y_pred[:,1])
#precision_score(y_test[:,10], y_pred[:,10])
#recall_score(y_test[:,10], y_pred[:,10])
# print(precision_score(y_test[:,10], y_pred[:,10]))
# print (recall_score(y_test[:,10], y_pred[:,10]))

Evaluating ...........
Labels: [0 1]
Confusion Matrix:
 [[4202  127]
 [ 812   68]]
Accuracy: 0.819735073911
Precision score: 0.348717948718
Recall score: 0.0772727272727


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [39]:
print(y_test.shape, y_pred.shape)

(5209, 36) (5209, 36)


In [40]:
print(classification_report(y_test[100], y_pred[100]))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36



In [41]:
### Trial code..........

# columns = df.iloc[:,4:].columns
# i = 0
# report = []
# for col in columns:
#     for i in range(y_pred.shape[1]):
#         #print(classification_report(y_test[i], y_pred[i],target_names =df.iloc[:,4:]))
#         report.append(classification_report(y_test[i], y_pred[i],target_names =[col]))
#         #print(classification_report(y_test[i], y_pred[i],target_names =[col]))
#         #i += 1
#         #print(i)
#         #print(*report,sep='\n',end=',')

# print(*report,sep='\n',end=',')

In [42]:
def display_classification_report(y_test,y_pred,df):
    
    """"Function to print multi-label multioutput metrics using sklearn.metrics.classification_report.
    Docstring-
    Input parameters:
    y_test- y testing set from training and testing split.
    y_pred- predictions results after model prediction.
    df - original clean dataframe with column names to extract 36 column names as target labels.

    Output - print display precision score, recall score & f1-score for target columns
    """
    # Loop to evaluate metrics per column.
    print ('Printing precision score, recall score & f1-score for target columns...............')
    columns = df.iloc[:,4:].columns
    i = 0
    report = []
    for col in columns:
        report.append(classification_report(y_test[:,i], y_pred[:,i],target_names =[col]))
        i+=1


    print(*report,sep='\n',end='\n')
    print ('..............................................Finished printing precision recall & f1-score for target columns.')
    
    
###Evaluate
print('Evaluating ...........')
display_classification_report(y_test,y_pred,df)

Evaluating ...........
Printing precision score, recall score & f1-score for target columns...............
             precision    recall  f1-score   support

    related       0.32      0.14      0.19      1212

avg / total       0.66      0.72      0.68      5209

             precision    recall  f1-score   support

    request       0.84      0.97      0.90      4329

avg / total       0.76      0.82      0.77      5209

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      5183

avg / total       0.99      1.00      0.99      5209

             precision    recall  f1-score   support

aid_related       0.59      0.79      0.67      3061

avg / total       0.52      0.55      0.52      5209

              precision    recall  f1-score   support

medical_help       0.92      0.99      0.96      4794

 avg / total       0.85      0.91      0.88      5209

                  precision    recall  f1-score   support

medical_products     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
def build_model():
    pipeline1 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    parameters = {
        'vect__ngram_range': ((1,1),(1, 2)),
        #'vect__max_df': [1.0],
        #'vect__min_df' : [1],
        #'tfidf__norm' : ['l2'],
        #'tfidf__use_idf' : ['True'],
        #'clf__estimator__min_samples_split': [2],
        #'clf__estimator__n_estimators': [100],
        #'clf__estimator__max_features': ['auto'],
        #'clf__estimator__criterion': ['gini'],
        #'clf__estimator__class_weight' : [{0: 1, 1: 1}]
    }
    
    
    cv = GridSearchCV(pipeline1, param_grid = parameters, verbose = 1, n_jobs=-1)
    
    return cv

In [32]:
# Train using GridSearch
print('Training using Grid Search.........')
start = time()

model1 = build_model()
model1.fit(X_train,y_train)

print('Time taken: {}'.format(time()-start))
print('...........................................finished training')

Training using Grid Search.........
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 36.6min finished


Time taken: 2484.850732088089
...........................................finished training


In [35]:
print('predicting using Grid Search.........')
start = time()
# Predict
y_pred1 = model1.predict(X_test)

print('Time taken: {}'.format(time()-start))
print('...........................................finished predicting')

predicting.........
Training using Grid Search.........
Time taken: 32.67859649658203
...........................................finished training


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [40]:
display_classification_report(y_test,y_pred1,df)

Printing precision score, recall score & f1-score for target columns...............
             precision    recall  f1-score   support

    related       0.33      0.16      0.22      1219

avg / total       0.67      0.72      0.68      5209

             precision    recall  f1-score   support

    request       0.83      0.98      0.90      4306

avg / total       0.76      0.82      0.77      5209

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      5188

avg / total       0.99      1.00      0.99      5209

             precision    recall  f1-score   support

aid_related       0.59      0.78      0.67      3049

avg / total       0.52      0.55      0.51      5209

              precision    recall  f1-score   support

medical_help       0.92      1.00      0.96      4810

 avg / total       0.86      0.92      0.89      5209

                  precision    recall  f1-score   support

medical_products       0.95      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
#from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier

In [13]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    #('clf', RidgeClassifierCV())
    ('clf',  MultiOutputClassifier(RandomForestClassifier()))
])
tuned_parameters = {
    'vect__ngram_range': ((1,1),(1, 2))
}


modelcv2 = GridSearchCV(pipeline2, param_grid = tuned_parameters, verbose = 1, n_jobs=-1)
modelcv2

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [18]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.5)

In [19]:
#Train using MultiOutputClassifier(RandomForestClassifier()) without TfID
modelcv2.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 17.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [43]:
#predict on test data
print('predicting using Grid Search.........')
start = time()

#Predict
y_pred2 = modelcv2.predict(X_test)


print('Time taken: {}'.format(time()-start))
print('...........................................finished predicting')

predicting using Grid Search.........
Time taken: 34.60213589668274
...........................................finished predicting


In [44]:
print ('best_estimator: {}'.format(modelcv2.best_estimator_),
       'best params:{}'.format(modelcv2.best_params_),
       'best score: {}'.format(modelcv2.best_score_),
       'Cross Validation results: {}'.format(modelcv2.cv_results_),
       'Scorer : {}'.format(modelcv2.scorer_),
       sep='\n' )

best_estimator: Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])
best params:{'vect__ngram_range': (1, 2)}
best score: 0.1794025036479533
Cross Validation results: {'mean_fit_time': array([  78.6131297 ,  111.36622723]), 'std_fit_time': array([ 0.1120005,  2.268008 ]), 'mean_score_time': array([ 26.37832721,  27.19716477]), 'std_score_time': array([ 0.34394588,  0.50386717]), 'param_vect__ngram_range': masked_array(data = [(1, 1) (1, 2)],
             mask = [False False],
       fill_value = ?)
, 'params': [{'vect__ngram_range': (1, 1)}, {'vect__ngram_range': (1, 2)}], 'split0_test_score': array([ 0.14674038,

In [45]:
display_classification_report(y_test,y_pred2,df)

Printing precision score, recall score & f1-score for target columns...............
             precision    recall  f1-score   support

    related       0.79      0.53      0.63      1212

avg / total       0.85      0.85      0.84      5209

             precision    recall  f1-score   support

    request       0.90      0.99      0.94      4329

avg / total       0.90      0.90      0.89      5209

             precision    recall  f1-score   support

      offer       1.00      1.00      1.00      5183

avg / total       1.00      1.00      1.00      5209

             precision    recall  f1-score   support

aid_related       0.74      0.95      0.83      3061

avg / total       0.80      0.77      0.76      5209

              precision    recall  f1-score   support

medical_help       0.95      1.00      0.97      4794

 avg / total       0.95      0.95      0.94      5209

                  precision    recall  f1-score   support

medical_products       0.97      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 1
  .format(len(labels), len(target_names))


### 9. Export your model as a pickle file

In [46]:
def save_model(model, model_filepath):
    # open the file for writing
    fileObject = open(model_filepath,'wb')
    # this writes the object a to the
    pickle.dump(model, fileObject)
    # # here we close the fileObject
    fileObject.close()

In [72]:
#trial code
# file_Name = "GridSearchModel.pkl"
# # open the file for writing
# fileObject = open(file_Name,'wb')
# # this writes the object a to the
# pickle.dump(cv2,fileObject)
# # # here we close the fileObject
# fileObject.close()

In [39]:
save_model(model1,'model1_pickle_file.pkl')

In [48]:
# we open the file for reading
fileObject = open('model1_pickle_file.pkl','rb')
# load the object from the file into var b
model1 = pickle.load(fileObject)

In [49]:
model1.best_score_

0.14078625258004129

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.